Run the line below (without the # symbol) to install pandas if not already installed

In [1]:
#!py -m pip install pandas

In [2]:
import requests
import csv
import os
import pandas as pd
import json
import re
import time
from datetime import datetime, timedelta
import math

In [218]:
input_path = "C:\\Users\\khan32\\Documents\\factgrid_py"

In [219]:
output_path = input_path

In [220]:
place_name = "Bamberg"

In [221]:
today_string = datetime.now().strftime('%Y-%m-%d')

In [222]:
# SELECT * FROM `role` r INNER JOIN url_external u on r.id = u.item_id where u.authority_id = 42 

Export the results from the query 
```sql
SELECT * FROM role r INNER JOIN url_external u on r.id = u.item_id where u.authority_id = 42
```  
from the main wiag database in the csv format.

Rename it to include the date. An example filename would be `role_2024_04_24.csv`

In [223]:
input_file = f"role_2024_05_08.csv"
input_path_file = os.path.join(input_path, input_file)
wiag_roles_df = pd.read_csv(input_path_file, names=['id', 'note', 'name', 'comment', 'gs_reg_id', 'generic_term', 'plural', 'definition', 'role_gp_fg_id', 'gender', 'lang', 'role_group_id','r_id','item_id','r_note','authority_id','role_fg_id','r_comment'])
len(wiag_roles_df)

96

In [224]:
input_file = f"WIAG-Domherren-DB-Ämter_2024-05-08_Bamberg.csv"
input_path_file = os.path.join(input_path, input_file)
role_all_df = pd.read_csv(input_path_file, sep=';')
len(role_all_df)

2773

In [225]:
last_modified = datetime.fromtimestamp(os.path.getmtime(input_file))
now = datetime.now()
assert last_modified.day == now.day and last_modified.month == now.month, f"The file was last updated on {last_modified.strftime('%d.%m')}"

### ERROR: If you get an error when you run the line above this means that the file was not updated today. 
A few solutions: 
* update the file again by downloading it again
* change the file name to something correct
* (not recommended) continue if you are sure that you need to use old data.

In [226]:
len(role_all_df)

2773

In [227]:
role_all_df.head()

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,date_begin,date_end,date_sort_key,GND,GSN,FactGrid
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,1016,1047,1016150,120912619,062-01186-001,Q653844
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,(1014 ?) 1015,1019,1019150,136969143,059-00710-001,Q652691
2,WIAG-Pers-CANON-24965-001,22325,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,1020,NaN,1020150,NaN,NaN,Q728753
3,WIAG-Pers-CANON-24965-001,31418,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,1020,NaN,1020150,NaN,NaN,Q728753
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,1057,1065,1057150,118699334,006-00007-001,Q652702


In [228]:
url = 'https://database.factgrid.de/sparql'
query = (
    """SELECT ?item ?gsn WHERE {
  ?item wdt:P471 ?gsn
}
"""
)
# SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

# make request: 
r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
factgrid_institution_df = pd.json_normalize(data['results']['bindings'])

len(factgrid_institution_df)

268

In [229]:
url = 'https://database.factgrid.de/sparql'
query = (
"""
SELECT ?item ?wiagid ?label ?alternative WHERE {
  ?item wdt:P2/wdt:P3* wd:Q164535.
  #?item schema:description ?itemDesc.
  ?item rdfs:label ?label.
  OPTIONAL {?item schema:description ?itemDesc.}
  OPTIONAL {?item skos:altLabel ?alternative. }
  OPTIONAL {?item wdt:P601 ?wiagid.}
  FILTER(LANG(?label) in ("en", "de"))
}
"""
)


# version 2
# SELECT ?item ?wiagid (group_concat(DISTINCT ?label; separator=',') as ?labels) (group_concat(DISTINCT ?itemDesc; separator=',') as ?itemDescs) (group_concat(DISTINCT ?alternative ; separator=',') as ?alternatives) WHERE {
#   ?item wdt:P2/wdt:P3* wd:Q164535.
#   ?item schema:description ?itemDesc.
#   ?item rdfs:label ?label.
#   OPTIONAL {?item schema:description ?itemDesc.}
#   OPTIONAL {?item skos:altLabel ?alternative. }
#   OPTIONAL {?item wdt:P601 ?wiagid.}
#   FILTER(LANG(?label) in ("en", "de"))
# }
# GROUP BY ?item ?wiagid

# SELECT ?item ?wiagid WHERE {
#   ?item wdt:P2/wdt:P3* wd:Q164535.
#   ?item wdt:P601 ?wiagid
# }
# SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

# make request: 
r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
factgrid_diocese_df = pd.json_normalize(data['results']['bindings'])

len(factgrid_diocese_df)

1290

In [230]:
url = 'https://database.factgrid.de/sparql'
query = (
"""
SELECT ?item ?label WHERE {
  ?item wdt:P2 wd:Q257052.
  ?item rdfs:label ?label.
  FILTER(LANG(?label) in ("de"))
}
"""
)

r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
factgrid_inst_roles_df = pd.json_normalize(data['results']['bindings'])

len(factgrid_inst_roles_df)

4342

In [231]:
# extract out q id
def extract_qid(df, column):
    df[column] = df[column].map(lambda x: x.strip('https://database.factgrid.de/entity/'))
 
#factgrid_df['item.value'] = factgrid_df['item.value'].map(lambda x: x.strip('https://database.factgrid.de/entity/'))

# drop irrelevant columns
def drop_type_columns(df):
    df.drop(columns=[column for column in df.columns if column.endswith('type')], inplace=True)
    df.drop(columns=[column for column in df.columns if column.endswith('xml:lang')], inplace=True)

In [232]:
extract_qid(factgrid_institution_df, 'item.value')
extract_qid(factgrid_diocese_df, 'item.value')
extract_qid(factgrid_inst_roles_df, 'item.value')

In [233]:
drop_type_columns(factgrid_institution_df)
drop_type_columns(factgrid_diocese_df)
drop_type_columns(factgrid_inst_roles_df)

In [234]:
# rename columns
factgrid_institution_df.columns = ['fg_institution_id', 'fg_gsn_id']
factgrid_diocese_df.columns = ["fg_diocese_id", "dioc_label", "dioc_alt", "dioc_wiag_id"]
factgrid_inst_roles_df.columns = ["fg_inst_role_id", "inst_role"]

In [235]:
# clean the diocese alts by removing BITECA and BETA entries 
factgrid_diocese_df['dioc_alt'].replace(['^BITECA.*', '^BETA.*'], '', regex=True, inplace=True)
factgrid_diocese_df
# set(factgrid_diocese_df['fg_alts'])

C:\Users\khan32\AppData\Local\Temp\ipykernel_13096\2196522043.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  factgrid_diocese_df['dioc_alt'].replace(['^BITECA.*', '^BETA.*'], '', regex=True, inplace=True)


,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id
0,Q153231,Bistum Meißen,Bishopric of Meißen,WIAG-Inst-DIOCGatz-017-001
1,Q153231,Diocese of Meißen,Bishopric of Meißen,WIAG-Inst-DIOCGatz-017-001
2,Q153231,Bistum Meißen,dioecesis Misnensis,WIAG-Inst-DIOCGatz-017-001
3,Q153231,Diocese of Meißen,dioecesis Misnensis,WIAG-Inst-DIOCGatz-017-001
4,Q153232,Bistum Merseburg,Diözese Merseburg,WIAG-Inst-DIOCGatz-018-001
...,...,...,...,...
1285,Q164666,Erzbistum Paris,NaN,NaN
1286,Q164667,Archdiocese of Reims,NaN,NaN
1287,Q164667,Archdiocese of Reims,NaN,NaN
1288,Q164668,Archdiocese of Rouen,NaN,NaN


In [236]:
factgrid_institution_df['fg_gsn_id'] = pd.to_numeric(factgrid_institution_df['fg_gsn_id'], downcast='float')

In [237]:
fg_gp = factgrid_institution_df.groupby('fg_gsn_id').count()
duplicate_fg_entries = factgrid_institution_df[factgrid_institution_df['fg_gsn_id'].isin(list(fg_gp[fg_gp['fg_institution_id'] > 1].index))]
assert duplicate_fg_entries.empty, f"There are possible duplicates on factgrid\n.{duplicate_fg_entries}"

In [238]:
# factgrid_institution_df.drop_duplicates(['fg_gsn_id'], inplace=True)

In [239]:
factgrid_institution_df.dtypes

fg_institution_id     object
fg_gsn_id            float32
dtype: object

In [240]:
factgrid_diocese_df.dtypes

fg_diocese_id    object
dioc_label       object
dioc_alt         object
dioc_wiag_id     object
dtype: object

In [241]:
role_all_df.dtypes

person_id            object
id                    int64
name                 object
role_group           object
role_group_en        object
role_group_fq_id     object
institution          object
institution_id      float64
diocese              object
diocese_id           object
date_begin           object
date_end             object
date_sort_key         int64
GND                  object
GSN                  object
FactGrid             object
dtype: object

In [242]:
# url = 'https://wiag-vocab.adw-goe.de/domherr/data'
# r = requests.get(url, params={'domstift': place_name})
# data = r.json()
# person_import_df = pd.json_normalize(data['persons'])

# print(len(person_import_df))
# person_import_df.head()

In [243]:
factgrid_institution_df.head()

,fg_institution_id,fg_gsn_id
0,Q422286,114.0
1,Q470546,119.0
2,Q633292,120.0
3,Q633339,131.0
4,Q633346,139.0


In [244]:
factgrid_diocese_df.head()

,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id
0,Q153231,Bistum Meißen,Bishopric of Meißen,WIAG-Inst-DIOCGatz-017-001
1,Q153231,Diocese of Meißen,Bishopric of Meißen,WIAG-Inst-DIOCGatz-017-001
2,Q153231,Bistum Meißen,dioecesis Misnensis,WIAG-Inst-DIOCGatz-017-001
3,Q153231,Diocese of Meißen,dioecesis Misnensis,WIAG-Inst-DIOCGatz-017-001
4,Q153232,Bistum Merseburg,Diözese Merseburg,WIAG-Inst-DIOCGatz-018-001


In [245]:
missing_institution_on_factgrid_df = role_all_df.merge(
    factgrid_institution_df, indicator = True, how='left', left_on='institution_id', right_on='fg_gsn_id', suffixes=('_wiag', '_institute_fg')
).loc[lambda x : x['_merge']!='both']

In [246]:
print(len(missing_institution_on_factgrid_df))
missing_institution_on_factgrid_df.head()

249


,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,date_begin,date_end,date_sort_key,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,_merge
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,1016,1047,1016150,120912619,062-01186-001,Q653844,NaN,NaN,left_only
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,(1014 ?) 1015,1019,1019150,136969143,059-00710-001,Q652691,NaN,NaN,left_only
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,1057,1065,1057150,118699334,006-00007-001,Q652702,NaN,NaN,left_only
7,WIAG-Pers-EPISCGatz-04423-001,4940,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Mainz,WIAG-Inst-DIOCGatz-016-001,1051,1059,1051150,136648185,007-02348-001,Q653060,NaN,NaN,left_only
9,WIAG-Pers-EPISCGatz-05263-001,82446,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,1085,1088,1085150,119063085,009-01109-001,Q656048,NaN,NaN,left_only


In [247]:
missing_institution_on_factgrid_df[['institution', 'institution_id', 'diocese', 'diocese_id']]

,institution,institution_id,diocese,diocese_id
0,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001
1,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001
4,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001
7,NaN,NaN,Erzbistum Mainz,WIAG-Inst-DIOCGatz-016-001
9,NaN,NaN,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001
...,...,...,...,...
2672,NaN,NaN,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001
2673,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001
2680,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001
2689,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001


In [248]:
missing_institution_diocese_on_factgrid_df = missing_institution_on_factgrid_df.merge(
    factgrid_diocese_df, indicator = '_second_merge', how='left', left_on='diocese_id', right_on='dioc_wiag_id', suffixes=('_wiag', '_dioc_fg')
).loc[lambda x : x['_second_merge']!='both']
missing_institution_diocese_on_factgrid_df.drop(['_merge','_second_merge'], axis=1, inplace=True)
null_entries = missing_institution_diocese_on_factgrid_df[missing_institution_diocese_on_factgrid_df[['institution_id', 'diocese_id', 'diocese']].isna().all(axis=1)]
null_entries = null_entries[null_entries['name'] != 'Kardinal'].sort_values('person_id')

# 4312 to 7979 should be removed
null_entries = null_entries[null_entries.person_id.str.contains('[0-7][0-9]{4}-[0-9]{3}$', regex=True)]
null_entries.to_csv(f'simple_null_entries_amter-{today_string}.csv', sep=';')

In [249]:
# lookup for the diocese by the diocese_id, then diocese labels, then diocese alts
def join_dioceses(input_df):
    rows = []
    for _, row in input_df.iterrows():
        search_row = pd.Series({'fg_diocese_id':None, 'dioc_label':None, 'dioc_alt':None, 'dioc_wiag_id':None})

        # NOTE: in the following three code blocks only the query changes.
        # this could be refactored somehow without running the query when defining it
        if not search_row['fg_diocese_id']:
            query = factgrid_diocese_df[factgrid_diocese_df['dioc_wiag_id'] == row['diocese_id']]
            if not query.empty:
                search_row = query.head(1).squeeze(axis=0)
    
        if not search_row['fg_diocese_id']:
            query = factgrid_diocese_df[factgrid_diocese_df['dioc_label'] == row['diocese']]
            if not query.empty:
                search_row = query.head(1).squeeze(axis=0)
    
        if not search_row['fg_diocese_id']:
            query = factgrid_diocese_df[factgrid_diocese_df['dioc_alt'] == row['diocese']]
            if not query.empty:
                search_row = query.head(1).squeeze(axis=0)
        rows.append(row.combine_first(search_row))
    
    dioc_joined_df = pd.DataFrame(rows)
    dioc_joined_df = dioc_joined_df[[*input_df.columns, *factgrid_diocese_df.columns]]
    
    return dioc_joined_df
dioc_joined_df = join_dioceses(missing_institution_on_factgrid_df)
dioc_joined_df

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,_merge,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,...,120912619,062-01186-001,Q653844,NaN,NaN,left_only,Q153244,Erzbistum Trier,Bistum Trier,WIAG-Inst-DIOCGatz-030-001
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,...,136969143,059-00710-001,Q652691,NaN,NaN,left_only,Q153220,Bistum Eichstätt,Diözese Eichstätt,WIAG-Inst-DIOCGatz-006-001
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,118699334,006-00007-001,Q652702,NaN,NaN,left_only,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001
7,WIAG-Pers-EPISCGatz-04423-001,4940,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Mainz,WIAG-Inst-DIOCGatz-016-001,...,136648185,007-02348-001,Q653060,NaN,NaN,left_only,Q153230,Erzbistum Mainz,Bistum Mainz,WIAG-Inst-DIOCGatz-016-001
9,WIAG-Pers-EPISCGatz-05263-001,82446,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,119063085,009-01109-001,Q656048,NaN,NaN,left_only,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,WIAG-Pers-EPISCGatz-12604-001,81521,Fürstbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,119510871,082-01201-001,Q323,NaN,NaN,left_only,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001
2673,WIAG-Pers-EPISCGatz-12604-001,81522,Koadjutor des Fürstbischofs,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,119510871,082-01201-001,Q323,NaN,NaN,left_only,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001
2680,WIAG-Pers-EPISCGatz-12604-001,81529,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,119510871,082-01201-001,Q323,NaN,NaN,left_only,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001
2689,WIAG-Pers-EPISCGatz-10590-001,128352,Generalvikar,Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,1202680992,076-01243-001,Q655909,NaN,NaN,left_only,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001


In [250]:
missing_institution_diocese_on_factgrid_df = dioc_joined_df[dioc_joined_df['fg_diocese_id'].isna()]

In [251]:
missing_institution_diocese_on_factgrid_df.drop(['_merge'], axis=1, inplace=True)
missing_institution_diocese_on_factgrid_df

C:\Users\khan32\AppData\Local\Temp\ipykernel_13096\1315841956.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_institution_diocese_on_factgrid_df.drop(['_merge'], axis=1, inplace=True)


,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,date_sort_key,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id
708,WIAG-Pers-EPISCGatz-05582-001,121601,Kardinal,Kardinal,Cardinal,Q254893,NaN,NaN,NaN,NaN,...,1439150,103078711,067-00130-001,Q164124,NaN,NaN,None,None,None,None


In [252]:
missing_institution_diocese_on_factgrid_df[missing_institution_diocese_on_factgrid_df.person_id.str.contains('[89][0-9]{4}-[0-9]{3}$', regex=True)]

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,date_sort_key,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id


In [253]:
# null_entries = missing_institution_diocese_on_factgrid_df[missing_institution_diocese_on_factgrid_df['institution_id'].isnull()][missing_institution_diocese_on_factgrid_df['diocese_id'].isnull()]
null_entries = missing_institution_diocese_on_factgrid_df[missing_institution_diocese_on_factgrid_df['name'] != 'Kardinal'].sort_values('person_id')
null_entries = null_entries[null_entries['role_group'] != 'Kurienamt']
null_entries = null_entries[null_entries['role_group'] != 'Papst']
null_entries = null_entries.sort_values(['diocese', 'institution'])

# 4312 to 7979 should be removed
null_entries = null_entries[null_entries.person_id.str.contains('[0-7][0-9]{4}-[0-9]{3}$', regex=True)]
null_entries

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,date_sort_key,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id


In [254]:
# TODO: sent ouput to file instead of stdout

for _, row in null_entries[null_entries['diocese'].notnull()][['diocese', 'diocese_id']].iterrows():
    print("CREATE")
    print('\t'.join(['LAST', 'Lde', f'"{row["diocese"]}"']))
    print('\t'.join(['LAST', 'P2', 'Q153166']))

    print('\t'.join(['LAST', 'P131', 'Q153178']))
    if type(row['diocese_id']) == str:
        print('\t'.join(['LAST', 'P601', f'"{row["diocese_id"]}"']))

In [255]:
# missing_institution_diocese_on_factgrid_df.to_csv('no_institution_or_dioc_on_fg_bamberg.csv', sep=';')

In [256]:
null_entries.to_csv(f'missing_amter-{today_string}_Bamberg.csv', sep=';')

### The missing institution/diocese would be outputted in a csv file above. Create them of factgrid and then re run this file to this point
### To continue below your csv files should be empty. The check below ensures that.

In [257]:
assert len(null_entries) == 0, "Create the entries on factgrid before continuing."

In [258]:
institution_joined_df = role_all_df.merge(
    factgrid_institution_df, how='left', left_on='institution_id', right_on='fg_gsn_id', suffixes=('_wiag', '_institute_fg')
)
institution_joined_df

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,date_begin,date_end,date_sort_key,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,1016,1047,1016150,120912619,062-01186-001,Q653844,NaN,NaN
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,(1014 ?) 1015,1019,1019150,136969143,059-00710-001,Q652691,NaN,NaN
2,WIAG-Pers-CANON-24965-001,22325,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,1020,NaN,1020150,NaN,NaN,Q728753,Q400557,3502.0
3,WIAG-Pers-CANON-24965-001,31418,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,1020,NaN,1020150,NaN,NaN,Q728753,Q400530,3492.0
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,1057,1065,1057150,118699334,006-00007-001,Q652702,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2768,WIAG-Pers-CANON-15191-001,128342,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,1799,NaN,1799150,NaN,054-02080-001,Q883390,Q400530,3492.0
2769,WIAG-Pers-CANON-15198-001,118309,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,1801,NaN,1801150,NaN,NaN,Q883392,Q400530,3492.0
2770,WIAG-Pers-CANON-15198-001,118311,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,1801,1803,1801150,NaN,NaN,Q883392,Q400557,3502.0
2771,WIAG-Pers-CANON-86198-001,53270,Domdekan,Leitungsamt Domstift,Dignitary of a cathedral chapter,Q648232,Domstift Bamberg,3492.0,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,um 1802,NaN,1802210,133862399,076-00865-001,Q883806,Q400530,3492.0


In [259]:
assert len(role_all_df) == len(institution_joined_df), "There may be duplicates on factgrid"

In [260]:
dioc_joined_df = join_dioceses(institution_joined_df)
dioc_joined_df

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,date_sort_key,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,...,1016150,120912619,062-01186-001,Q653844,NaN,NaN,Q153244,Erzbistum Trier,Bistum Trier,WIAG-Inst-DIOCGatz-030-001
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,...,1019150,136969143,059-00710-001,Q652691,NaN,NaN,Q153220,Bistum Eichstätt,Diözese Eichstätt,WIAG-Inst-DIOCGatz-006-001
2,WIAG-Pers-CANON-24965-001,22325,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,...,1020150,NaN,NaN,Q728753,Q400557,3502.0,None,None,None,None
3,WIAG-Pers-CANON-24965-001,31418,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,1020150,NaN,NaN,Q728753,Q400530,3492.0,None,None,None,None
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,1057150,118699334,006-00007-001,Q652702,NaN,NaN,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2768,WIAG-Pers-CANON-15191-001,128342,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,1799150,NaN,054-02080-001,Q883390,Q400530,3492.0,None,None,None,None
2769,WIAG-Pers-CANON-15198-001,118309,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,1801150,NaN,NaN,Q883392,Q400530,3492.0,None,None,None,None
2770,WIAG-Pers-CANON-15198-001,118311,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,...,1801150,NaN,NaN,Q883392,Q400557,3502.0,None,None,None,None
2771,WIAG-Pers-CANON-86198-001,53270,Domdekan,Leitungsamt Domstift,Dignitary of a cathedral chapter,Q648232,Domstift Bamberg,3492.0,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,1802210,133862399,076-00865-001,Q883806,Q400530,3492.0,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001


In [261]:
print(len(wiag_roles_df))
wiag_roles_df.head()
#wiag_roles_df[wiag_roles_df['name'] == '']

96


,id,note,name,comment,gs_reg_id,generic_term,plural,definition,role_gp_fg_id,gender,lang,role_group_id,r_id,item_id,r_note,authority_id,role_fg_id,r_comment
0,3,NaN,Administrator des Bistums,NaN,NaN,NaN,NaN,Ein Bistumsadministrator (auch: Diözesanadmini...,Q648236,männlich,de,33,1298102,3,NaN,42,Q902170,NaN
1,4,NaN,Administrator des Erzbistums,NaN,NaN,NaN,NaN,NaN,Q648236,männlich,de,33,1298104,4,NaN,42,Q902170,NaN
2,5,NaN,Administrator des Fürstbischofs,NaN,NaN,NaN,NaN,Ein Bistumsadministrator (auch: Diözesanadmini...,Q648236,männlich,de,33,1298982,5,NaN,42,Q902170,NaN
3,9,NaN,Apostolischer Administrator,NaN,NaN,NaN,NaN,Ist ein Bischofsstuhl vakant oder bestehen ern...,Q648236,männlich,de,33,1298980,9,NaN,42,Q38958,NaN
4,13,NaN,Bischofsadministrator,NaN,NaN,NaN,NaN,Ein Bistumsadministrator (auch: Diözesanadmini...,Q648236,männlich,de,33,1298106,13,NaN,42,Q902170,NaN


In [262]:
joined_df = dioc_joined_df.merge(
    wiag_roles_df[['name', 'role_fg_id']], how='left', left_on='name', right_on='name', suffixes=('_wiag', '_institute_fg')
)
print(len(joined_df))
joined_df.head()

2782


,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id,role_fg_id
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,...,120912619,062-01186-001,Q653844,NaN,NaN,Q153244,Erzbistum Trier,Bistum Trier,WIAG-Inst-DIOCGatz-030-001,Q172539
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,...,136969143,059-00710-001,Q652691,NaN,NaN,Q153220,Bistum Eichstätt,Diözese Eichstätt,WIAG-Inst-DIOCGatz-006-001,Q38809
2,WIAG-Pers-CANON-24965-001,22325,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,...,NaN,NaN,Q728753,Q400557,3502.0,None,None,None,None,Q38837
3,WIAG-Pers-CANON-24965-001,31418,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,NaN,Q728753,Q400530,3492.0,None,None,None,None,Q38837
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,118699334,006-00007-001,Q652702,NaN,NaN,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001,Q38809


In [263]:
joined_df[joined_df['role_fg_id'].isna()]

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id,role_fg_id
430,WIAG-Pers-CANON-88937-001,64229,Kanonikatsbewerber,NaN,NaN,NaN,"Stift Haug, Würzburg",3428.0,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,NaN,036-02711-001,Q883846,Q400535,3428.0,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001,NaN
591,WIAG-Pers-CANON-88928-001,63313,Kanonikatsbewerber,NaN,NaN,NaN,"Kollegiatstift Neumünster, Würzburg",1058.0,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,NaN,036-02244-001,Q883844,Q400568,1058.0,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001,NaN
667,WIAG-Pers-CANON-88968-001,63363,Kanonikatsbewerber,NaN,NaN,NaN,"Kollegiatstift Neumünster, Würzburg",1058.0,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,NaN,036-04359-001,Q883854,Q400568,1058.0,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001,NaN
682,WIAG-Pers-CANON-88947-001,63306,Kanonikatsbewerber,NaN,NaN,NaN,"Kollegiatstift Neumünster, Würzburg",1058.0,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,NaN,036-02798-001,Q883848,Q400568,1058.0,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001,NaN
694,WIAG-Pers-CANON-88957-001,63364,Kanonikatsbewerber,NaN,NaN,NaN,"Kollegiatstift Neumünster, Würzburg",1058.0,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,NaN,036-03573-001,Q883852,Q400568,1058.0,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001,NaN
721,WIAG-Pers-CANON-88903-001,63318,Kanonikatsbewerber,NaN,NaN,NaN,"Kollegiatstift Neumünster, Würzburg",1058.0,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,NaN,036-01231-001,Q883840,Q400568,1058.0,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001,NaN
778,WIAG-Pers-CANON-88938-001,63309,Kanonikatsbewerber,NaN,NaN,NaN,"Kollegiatstift Neumünster, Würzburg",1058.0,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,NaN,036-02726-001,Q883847,Q400568,1058.0,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001,NaN
851,WIAG-Pers-CANON-88953-001,63365,Kanonikatsbewerber,NaN,NaN,NaN,"Kollegiatstift Neumünster, Würzburg",1058.0,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,NaN,036-03496-001,Q883850,Q400568,1058.0,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001,NaN
862,WIAG-Pers-CANON-88974-001,63361,Kanonikatsbewerber,NaN,NaN,NaN,"Kollegiatstift Neumünster, Würzburg",1058.0,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,NaN,036-04551-001,Q883855,Q400568,1058.0,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001,NaN
876,WIAG-Pers-CANON-88901-001,63319,Kanonikatsbewerber,NaN,NaN,NaN,"Kollegiatstift Neumünster, Würzburg",1058.0,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,NaN,036-01114-001,Q883839,Q400568,1058.0,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001,NaN


In [264]:
# ignore all Kanonikatsbewerber and Vikariatsbewerber offices
joined_df = joined_df[~joined_df['name'].isin(['Vikariatsbewerber', 'Kanonikatsbewerber'])]

In [265]:
joined_df[joined_df['role_fg_id'].isna()][['person_id', 'name']]

,person_id,name
2623,WIAG-Pers-EPISCGatz-10668-001,Kapitularvikar


In [266]:
joined_df[joined_df['FactGrid'].isna()]

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id,role_fg_id
65,WIAG-Pers-CANON-80982-001,56198,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,NaN,007-01739-001,NaN,Q400530,3492.0,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001,Q38837
1225,WIAG-Pers-CANON-45781-001,139446,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,NaN,NaN,Q400530,3492.0,None,None,None,None,Q38837
1226,WIAG-Pers-CANON-45781-001,139448,Domcellerar,Leitungsamt Domstift,Dignitary of a cathedral chapter,Q648232,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,NaN,NaN,Q400530,3492.0,None,None,None,None,Q893490
1227,WIAG-Pers-CANON-45781-001,139450,Domkustos,Leitungsamt Domstift,Dignitary of a cathedral chapter,Q648232,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,NaN,NaN,Q400530,3492.0,None,None,None,None,Q902176
1228,WIAG-Pers-CANON-45781-001,139452,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Augsburg,3498.0,NaN,NaN,...,NaN,NaN,NaN,Q898020,3498.0,None,None,None,None,Q38837
2285,WIAG-Pers-CANON-26451-001,139416,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,NaN,NaN,Q400530,3492.0,None,None,None,None,Q38837
2286,WIAG-Pers-CANON-26451-001,139418,Domizellar,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,...,NaN,NaN,NaN,Q400557,3502.0,None,None,None,None,Q38838
2287,WIAG-Pers-CANON-26451-001,139420,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Worms,3490.0,NaN,NaN,...,NaN,NaN,NaN,Q898042,3490.0,None,None,None,None,Q38837
2376,WIAG-Pers-CANON-15070-001,139318,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,140187669,054-03017-001,NaN,Q400530,3492.0,None,None,None,None,Q38837


In [268]:
joined_df[joined_df['FactGrid'].isna()]['person_id'].unique()

array(['WIAG-Pers-CANON-80982-001', 'WIAG-Pers-CANON-45781-001',
       'WIAG-Pers-CANON-26451-001', 'WIAG-Pers-CANON-15070-001'],
      dtype=object)

In [269]:
joined_df[joined_df['FactGrid'].isna()].to_csv(f'missing_factgrid-{today_string}_Bamberg.csv', sep=';')

In [270]:
assert joined_df[joined_df['FactGrid'].isna()].empty, "There are missing persons on factgrid"

AssertionError: There are missing persons on factgrid

In [271]:
factgrid_inst_roles_df

,fg_inst_role_id,inst_role
0,Q172283,Generalvikar der Diözese Merseburg
1,Q172284,Generalvikar der Diözese Minden
2,Q172285,Generalvikar der Diözese Münster
3,Q172286,Generalvikar der Diözese Naumburg
4,Q172287,Generalvikar der Diözese Osnabrück
...,...,...
4337,Q450052,Generalsekretär des HWPP Stadtkomitees Szekszárd
4338,Q450053,Generalsekretär des HWPP-Stadtkomitees Szolnok
4339,Q450054,Generalsekretär des Komitees des Kreises HWPP ...
4340,Q450055,Generalsekretär des HWPP Szombathely City Comm...


In [272]:
factgrid_inst_roles_df[factgrid_inst_roles_df['inst_role'].str.contains('Erzbistum Trier')]

,fg_inst_role_id,inst_role
4032,Q908232,Archidiakon Erzbistum Trier


In [306]:
# add factgrid ids for roles
found = 0
data_dict = {}
not_found = []
dupl = {}
for i, (name, inst, dioc) in joined_df[['name', 'institution', 'diocese']].iterrows():
    if name == "Kardinal":
        # this is okay
        # manually add qid for kardinals Q254893
        data_dict[i] = "Q254893"
        continue
    search_res = pd.DataFrame()
    if pd.isna(inst):
    # if name in ["Bischof", "Fürstbischof", "Ernannter Bischof", "Erzbischof"]:
        if pd.isna(dioc):
            print(i, name, inst, dioc)
        if name not in ["Archidiakon", "Koadjutor"]:
            dioc = dioc.lstrip('Bistum').lstrip('Erzbistum').lstrip('Patriarchat').lstrip()
        if name == "Fürstbischof" and dioc in ["Passau", "Straßburg"]:
            name = "Bischof"    
        search_res = factgrid_inst_roles_df[factgrid_inst_roles_df['inst_role'].str.contains(f"^{name}.*{dioc}")]
        if name == "Erzbischof" and dioc == "Salzburg":
            # will be merged in later
            search_res = factgrid_inst_roles_df[factgrid_inst_roles_df['fg_inst_role_id'] == 'Q172567']
    else:
        name = name.replace('Domkanoniker', 'Domherr')
        search_res = factgrid_inst_roles_df[factgrid_inst_roles_df['inst_role'] == f"{name} {inst}"]
    if len(search_res) == 1:
        found += 1
        data_dict[i] = search_res['fg_inst_role_id'].values[0]
    elif len(search_res) >= 2:
        # print("+" * 10)
        print(name, inst, dioc)
        print(search_res)
        print()
        dupl[i] = (name, inst, dioc, search_res)
    elif len(search_res) == 0:
        not_found.append((name, inst, dioc))
        print(name, inst, dioc)
print(found, len(dupl), len(not_found))

Propst Kollegiatstift St. Nikolaus, Spalt nan
Domsenior Domstift Würzburg nan
Cellerar Domstift Eichstätt nan
Propst Benediktinerkloster, später Kollegiatstift St. Cyriakus, Wiesensteig nan
Domscholaster Domstift Eichstätt nan
Domcellerar Domstift Augsburg nan
Kapitularvikar nan Würzburg
Propst Kollegiatstift Unsere Liebe Frau, Eichstätt nan
2760 0 8


In [307]:
final_joined_df = joined_df.merge(pd.Series(data_dict).rename('fg_inst_role_id'), left_index=True, right_index=True)
print(len(final_joined_df))
final_joined_df.head()

2761


,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id,role_fg_id,fg_inst_role_id
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,...,062-01186-001,Q653844,NaN,NaN,Q153244,Erzbistum Trier,Bistum Trier,WIAG-Inst-DIOCGatz-030-001,Q172539,Q172572
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,...,059-00710-001,Q652691,NaN,NaN,Q153220,Bistum Eichstätt,Diözese Eichstätt,WIAG-Inst-DIOCGatz-006-001,Q38809,Q172630
2,WIAG-Pers-CANON-24965-001,22325,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,...,NaN,Q728753,Q400557,3502.0,None,None,None,None,Q38837,Q390632
3,WIAG-Pers-CANON-24965-001,31418,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,Q728753,Q400530,3492.0,None,None,None,None,Q38837,Q400601
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,006-00007-001,Q652702,NaN,NaN,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001,Q38809,Q195266


In [309]:
# https://database.factgrid.de/query/embed.html#SELECT%20%3FPropertyLabel%20%3FProperty%20%3FPropertyDescription%20%3Freciprocal%20%3FreciprocalLabel%20%3Fexample%20%3Fuseful_statements%20%3Fwd%20WHERE%20%7B%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%22.%20%7D%0A%20%20%3FProperty%20wdt%3AP8%20wd%3AQ77483.%0A%20%20OPTIONAL%20%7B%20%3FProperty%20wdt%3AP364%20%3Fexample.%20%7D%0A%20%20OPTIONAL%20%7B%20%3FProperty%20wdt%3AP86%20%3Freciprocal.%20%7D%0A%20%20OPTIONAL%20%7B%20%3FProperty%20wdt%3AP343%20%3Fwd.%20%7D%0A%20%20OPTIONAL%20%7B%20%3FProperty%20wdt%3AP310%20%3Fuseful_statements.%20%7D%0A%7D%0AORDER%20BY%20%3FPropertyLabel

def format_datetime(entry: datetime, resolution):
    julian_ending = "/J" if entry.year < 1582 else ""
    return f"+{entry.isoformat()}Z/{resolution}" + julian_ending

def date_parsing(date_string: str, end=False):
    return_property = "P50" if end else "P49"
    qualifier = None
    entry = None
    resolution = 7
    
    if matches := re.match(r'frühestens (\d{3,4})', date_string):
        return_property = "P1125" if end else "P1126"
        entry = datetime(int(matches.group(1)), 1, 1)
        resolution = 9
    
    elif matches := re.match(r'(kurz )?vor (\d{3,4})', date_string):
        return_property = "P1123" if end else "P1124"
        if matches.group(1):
            qualifier = ("P788 " if end else "P787 ") + date_string
        entry = datetime(int(matches.group(2)), 1, 1)
        resolution = 9
        
    elif matches := re.match(r'(kurz )?nach (\d{3,4})', date_string):
        return_property = "P1125" if end else "P1126"
        if matches.group(1):
            qualifier = ("P788 " if end else "P787 ") + date_string
        entry = datetime(int(matches.group(2)), 1, 1)
        resolution = 9
        
    elif matches := re.match(r'(\d{1,2})\. Jahrhundert', date_string):
        century = int(matches.group(1))
        entry = datetime(100 * (century), 1, 1)
    
    elif matches := re.match(r'(\d)\. Hälfte des (\d{1,2})\. Jhs\.', date_string):
        half = int(matches.group(1))
        year = int(matches.group(2)) - 1
        latest_year   = year * 100 + (half * 50)
        earliest_year = latest_year - 50 + 1
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        qualifier = ("P788 " if end else "P787 ") + date_string
    
    elif matches := re.match(r'(\w+) Viertel des (\d{1,2})\. Jhs\.', date_string):
        number_map = {
            "erstes":  1,
            "zweites": 2,
            "drittes": 3,
            "viertes": 4,
        }
        quarter = number_map[matches.group(1)]
        year    = int(matches.group(2)) - 1
        latest_year   = year * 100 + (quarter * 25)
        earliest_year = latest_year - 25 + 1
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        qualifier = ("P788 " if end else "P787 ") + date_string

    elif matches := re.match(r'frühes (\d{1,2})\. Jh\.', date_string):
        year = int(matches.group(1)) - 1
        latest_year   = year * 100 + 20
        earliest_year = latest_year - 20 + 1
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        qualifier = ("P788 " if end else "P787 ") + date_string

    elif matches := re.match(r'spätes (\d{1,2})\. Jh\.', date_string):
        year = int(matches.group(1))
        latest_year   = year * 100
        earliest_year = latest_year - 20 + 1
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        qualifier = ("P788 " if end else "P787 ") + date_string

    elif matches := re.match(r'(Anfang|Mitte|Ende) (\d{1,2})\. Jh\.', date_string):
        number_map = {
            "Anfang":  1,
            "Mitte": 2,
            "Ende": 3,
        }
        third = number_map[matches.group(1)]
        year = int(matches.group(2)) - 1
        latest_year   = year * 100 + (third * 33)
        earliest_year = latest_year - 33 + 1
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        qualifier = ("P788 " if end else "P787 ") + date_string

    elif matches := re.match(r'(ca\.|um) (\d{3,4})', date_string):
        year = int(matches.group(2))
        latest_year   = year + 5
        earliest_year = latest_year - 10
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        resolution = 9
        qualifier = ("P785 " if end else "P786 ") + "Q10"

    elif matches := re.match(r'(\d{3,4})er Jahre', date_string):
        year = int(matches.group(1))
        latest_year   = year + 10
        earliest_year = latest_year - 10 + 1
        entry = datetime(year, 1, 1)
        resolution = 8
    
    elif matches := re.match(r'Wende zum (\d{1,2})\. Jh\.', date_string):
        year = int(matches.group(1)) - 1
        latest_year   = year * 100 + 10
        earliest_year = latest_year - 20 + 1
        entry = datetime(latest_year, 1, 1)
        qualifier = ("P788 " if end else "P787 ") + date_string

    elif matches := re.match(r'Anfang der (\d{3,4})er Jahre', date_string):
        year = int(matches.group(1))
        latest_year   = year + 3
        earliest_year = year
        entry = datetime(year, 1, 1)
        resolution = 8
        qualifier = ("P788 " if end else "P787 ") + date_string

    elif matches := re.match(r'\((\d{3,4}) \?\) (\d{3,4})', date_string):
        year1 = matches.group(1)
        year2 = matches.group(2)
        entry = datetime(year2, 1, 1)
        resolution = 9
        qualifier = ("P788 " if end else "P787 ") + date_string
    
    elif matches := re.match(r'(\d{3,4})/(\d{3,4})', date_string):
        year1 = int(matches.group(1))
        year2 = int(matches.group(2))

        if year2 - year1 == 1:
            # check for consecutive years
            qualifier = "P786 = Q912616" if end else "P785 = Q912616"
        entry = datetime(year1, 1, 1)
        resolution = 9

    elif matches := re.match(r'(\d{3,4})\?', date_string):
        year = matches.group(1)
        entry = datetime(year, 1, 1)
        resolution = 9
        qualifier = ("P786 " if end else "P785 ") + date_string
    
    elif matches := re.match(r'(\d{3,4})', date_string):
        year = int(matches.group(1))
        entry = datetime(year, 1, 1)
        resolution = 9
    else:
        raise Exception("Couldn't parse date")
        
    if qualifier:
        return (return_property, format_datetime(entry, resolution), qualifier)
    else:
        return (return_property, format_datetime(entry, resolution))

tests = {
    "1205": "+1205-01-01T00:00:00Z/9/J",
    "12. Jahrhundert": "+1200-01-01T00:00:00Z/7/J",
    "1. Hälfte des 12. Jhs.": "+1125-01-01T00:00:00Z/7/J",
    "2. Hälfte des 12. Jhs.": "+1175-01-01T00:00:00Z/7/J",
    "erstes Viertel des 12. Jhs.": "+1113-01-01T00:00:00Z/7/J",
    "zweites Viertel des 12. Jhs.": "+1138-01-01T00:00:00Z/7/J",
    "drittes Viertel des 12. Jhs.": "+1163-01-01T00:00:00Z/7/J",
    "viertes Viertel des 12. Jhs.": "+1188-01-01T00:00:00Z/7/J",
    "frühes 12. Jh.": "+1110-01-01T00:00:00Z/7/J",
    "spätes 12. Jh.": "+1190-01-01T00:00:00Z/7/J",
    "Anfang 12. Jh.": "+1117-01-01T00:00:00Z/7/J",
    "Mitte 12. Jh.": "+1150-01-01T00:00:00Z/7/J",
    "Ende 12. Jh.": "+1183-01-01T00:00:00Z/7/J",
    "ca. 1050": "+1050-01-01T00:00:00Z/9/J",
    "um 1050": "+1050-01-01T00:00:00Z/9/J",
    "1230er Jahre": "+1230-01-01T00:00:00Z/8/J",
    "Wende zum 12. Jh.": "+1110-01-01T00:00:00Z/7/J",
    "Anfang der 1480er Jahre": "+1480-01-01T00:00:00Z/8/J",
    "frühestens 1342": "+1342-01-01T00:00:00Z/9/J",
    "vor 1230": "+1230-01-01T00:00:00Z/9/J",
    "nach 1230": "+1230-01-01T00:00:00Z/9/J",
    "kurz vor 1200": "+1200-01-01T00:00:00Z/9/J",
    "kurz nach 1200": "+1200-01-01T00:00:00Z/9/J",
    "1164/1165": "+1164-01-01T00:00:00Z/9/J",
    "1164/1177": "+1164-01-01T00:00:00Z/9/J",
}

for key, value in tests.items():
    retval = date_parsing(key)[1]
    assert retval == value, f"{key}: Returned {retval} instead of {value}"


In [92]:
# create roles for people on factgrid
for _, row in final_joined_df.iterrows():
    print(row)
    # FIXME: role_fg_id to be replace by the fg id for something like "Domherr Domstift Bamberg"
    print('\t'.join([row['FactGrid'], 'P165', row['role_fg_id'], date_parsing(row['date_begin']), date_parsing(row['date_end'])]))
    print('\t'.join([row['FactGrid'], date_parsing(row['date_begin'])]))
    print('\t'.join([row['FactGrid'], date_parsing(row['date_end'])]))
    
    # Determine correct type of career statement
    # vs https://database.factgrid.de/wiki/Item:Q257052 (complete list https://database.factgrid.de/wiki/Special:WhatLinksHere/Item:Q394330)
    print('\t'.join([row['FactGrid'], 'P2', 'Q37073']))

    print()
    break

person_id            WIAG-Pers-EPISCGatz-05072-001
id                                            5154
name                                    Erzbischof
role_group            Oberstes Leitungsamt Diözese
role_group_en             Head of an (arch)diocese
role_group_fq_id                           Q648236
institution                                    NaN
institution_id                                 NaN
diocese                            Erzbistum Trier
diocese_id              WIAG-Inst-DIOCGatz-030-001
date_begin                                    1016
date_end                                      1047
date_sort_key                              1016150
GND                                      120912619
GSN                                  062-01186-001
FactGrid                                   Q653844
fg_institution_id                              NaN
fg_gsn_id                                      NaN
fg_diocese_id                              Q153244
dioc_label                     

ValueError: time data '1016' does not match format 'nach %Y'

In [206]:
role_all_df[role_all_df['name'].str.contains('Bene')]['name'].iloc[0]

'Benefiziat'

In [ ]:
joined_

In [45]:
# all_roles = set()

# def add_possible_list(a_set: set, element):
#     if type(element) == list:
#         # add all items in list
#         for item in element:
#             a_set.add(item)
#     else:
#         # simply add the given element
#         a_set.add(element)

# for summary in missing_on_factgrid_df['summary_offices'].tolist():
#     print(summary)
#     offices = re.split(r'(\d+),', summary)
#     offices = [office.lstrip() for office in offices]
#     new_offices = offices
#     for index, office in enumerate(offices):
#         if re.match('\d+', office):
#             new_offices[index - 1] += office
#             new_offices.pop(index)
#     print(new_offices)
#     for office in new_offices:
#         for office_name in re.match(r'\w+(, \w+)*', office).group().split(','):
#             office_name = office_name.strip()
#             print(office_name)
#             all_roles.add(office_name)
#     print()

# print('#'*10)
# print(all_roles)

# url = 'https://database.factgrid.de/sparql'
# query = (
#     f"""SELECT DISTINCT ?item ?label
# WHERE
# {{
#   ?item wdt:P2 wd:Q37073;
#         rdfs:label ?label.
#   FILTER(LANG(?label) = "de").
#   FILTER REGEX (?label, "({'|'.join(list(all_roles))})$").
# }}
# ORDER BY ?label  
# """
# )
# print(query)
# # SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

# # make request: 
# r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
# data = r.json()
# factgrid_roles_df = pd.json_normalize(data['results']['bindings'])

# len(factgrid_roles_df)

# list(all_roles)

In [46]:
extract_qid(factgrid_roles_df, 'item.value')
factgrid_roles_df

,item.type,item.value,label.xml:lang,label.type,label.value
0,uri,Q38837,de,literal,Domherr
1,uri,Q38838,de,literal,Domizellar
2,uri,Q38842,de,literal,Dompropst
3,uri,Q38843,de,literal,Domvikar
4,uri,Q43669,de,literal,Stadtdiakon/Vikar
5,uri,Q173806,de,literal,Vize-Propst


In [47]:
role_all_df

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,date_begin,date_end,date_sort_key,GND,GSN,FactGrid
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,1016,1047,1016150,120912619,062-01186-001,Q653844
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,(1014 ?) 1015,1019,1019150,136969143,059-00710-001,Q652691
2,WIAG-Pers-CANON-24965-001,22325,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,1020,NaN,1020150,NaN,NaN,Q728753
3,WIAG-Pers-CANON-24965-001,31418,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,1020,NaN,1020150,NaN,NaN,Q728753
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,1057,1065,1057150,118699334,006-00007-001,Q652702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,WIAG-Pers-CANON-85544-001,51267,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,um 1800,NaN,1800210,NaN,076-01245-001,NaN
2524,WIAG-Pers-CANON-15198-001,118309,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,1801,NaN,1801150,NaN,NaN,NaN
2525,WIAG-Pers-CANON-15198-001,118311,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,1801,1803,1801150,NaN,NaN,NaN
2526,WIAG-Pers-CANON-86198-001,53270,Domdekan,Leitungsamt Domstift,Dignitary of a cathedral chapter,Q648232,Domstift Bamberg,3492.0,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,um 1802,NaN,1802210,133862399,076-00865-001,NaN


In [52]:
role_all_df[role_all_df['person_id'] == 'WIAG-Pers-EPISCGatz-05072-001']

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,date_begin,date_end,date_sort_key,GND,GSN,FactGrid
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,1016,1047,1016150,120912619,062-01186-001,Q653844


In [55]:
role_all_df[role_all_df[]]

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,date_begin,date_end,date_sort_key,GND,GSN,FactGrid
person_id,,,,,,,,,,,,,,,,
False,2522,2522,2522,2500,2498,2498,2284,2284,728,727,2514,1623,2522,1166,2009,1296
True,6,6,6,6,6,6,6,6,0,0,6,4,6,6,6,0


In [ ]:
# FIXME: fix this tentative mapping after discussion

office_mapping = {
    'Domherr': 'Domherr',
    '':''
}

In [30]:
# for summary in missing_on_factgrid_df['summary_offices'].tolist():
#     print(summary)